In [1]:
import sys
sys.path.append(r"D:\work_dir\stock\src\stockProject")
import akshare as ak
import tushare as ts
from datetime import datetime,timedelta
import pandas as pd
from pprint import pprint
import download_data
import pysnooper,time
from tqdm.autonotebook import tqdm
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

c:\users\wuwukai\envs\h1\lib\site-packages\ipykernel_launcher.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  # Remove the CWD from sys.path while we load stuff.


In [2]:

def get_ma_gt(df,st,en,add_day,line_count):
    buf = df.loc[st:en].diff().fillna(0)
    res = 0
    for i in df.columns:
        add = buf[buf[i]>0].ma5.count()
        if i in ['ma10','ma20'] and add < 4:
            res = 0;break
        if add >= add_day :
            res+=1
    return res >= line_count

In [12]:
def main(df):
    col = ['ma5', 'ma10', 'ma20', 'ma30',]
    var = 0.06  # 方查系数
    add_day = 4  # 增长天数
    line_count = 2  # 均线增长数量
    result = []
    res = []
    tq = tqdm(range(len(set(df.symbol))))
    for index,value in df.groupby(df.symbol):
        ss = time.time()
        value['ma30'] = value['close'].rolling(30,min_periods=1).mean()
#         value['ma60'] = value['close'].rolling(60,min_periods=1).mean()
        _df = value[col]
        min_date = _df.var(axis=1)
        min_date = min_date[min_date < var ].sort_index()
        y = 0
        for date in min_date.index:
            if res and res[-1].month == date.month and res[-1].year == date.year:
                continue
            st = (date  + timedelta(days=-12)).date().strftime('%Y-%m-%d')
            y+=1
            if get_ma_gt(_df,st,str(date.date()),add_day,line_count):
                res.append(date)
        df2 = pd.DataFrame(res,columns=['jx_date'])
        df2['symbol'],df2['name'] = index,value.name[0]
        v = value.loc[res,['pe','close']]
        df2['close'] = v.close.values
        df2['pe'] = v.pe.values
        result.append(df2)
        res.clear()
        tq.update(1)

    return result
    # print(time.time() - ss)

In [13]:
df = download_data.main(update=False)
df.sort_values(['symbol','date'],inplace=True)
df.set_index('date',inplace=True)
df.index = pd.to_datetime(df.index)

[2020-11-01 21:15:14,010][INFO][download_data.py:51] stock 数据更新


In [16]:
main(df.query("name=='恩捷股份'"))[0]

,jx_date,symbol,name,close,pe


In [110]:
df3.query("symbol=='002373'")

,jx_date,symbol,name,close,pe
0,2019-12-12,002373,千方科技,17.22,35.71
1,2020-04-29,002373,千方科技,22.52,35.71
2,2020-05-22,002373,千方科技,22.33,35.71
3,2020-06-26,002373,千方科技,22.98,35.71
4,2020-10-21,002373,千方科技,22.97,35.71


In [69]:
df = pd.read_pickle(r"D:\work_dir\stock\src\stockProject\results\ma60_get_ma_B_2020-11-01.pkl")

In [81]:
df.name.iloc[0]

'平安银行'

In [72]:
df.query("t_date>'2020-10-10' and jc_counts>=1").drop_duplicates('symbol')

,jx_date,jx_names,jx_counts,var,jc_counts,symbol,t_date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,name,industry,pe,ma30,ma60
8,2020-10-19,"ma5,ma10,",2,0.002195,2,000017,2020-10-19,2.53,2.53,2.51,2.51,12773.00,-0.01,-0.40,2.528,2.496,2.496,18256.80,18537.73,19469.49,0.42,*ST中华A,文教休闲,194.84,2.528667,2.610500
10,2020-10-16,"ma5,ma10,ma20,",3,0.013713,1,000027,2020-10-16,5.79,5.89,5.81,5.79,244370.19,0.02,0.34,5.836,5.651,5.549,301550.21,247193.16,175273.16,0.51,深圳能源,火力发电,4.96,5.553000,5.618167
9,2020-10-23,"ma5,ma10,",2,0.010097,1,000056,2020-10-23,3.45,3.48,3.48,3.40,50834.00,0.01,0.29,3.456,3.442,3.420,54305.93,65406.12,72698.80,0.56,皇庭国际,房产服务,95.96,3.464667,3.667167
7,2020-10-16,"ma5,ma10,",2,0.020087,1,000065,2020-10-16,7.72,7.75,7.72,7.67,12652.79,-0.02,-0.26,7.756,7.639,7.643,20752.20,17481.57,20309.57,0.19,北方国际,建筑工程,10.35,7.747333,7.988000
4,2020-10-20,"ma5,ma10,ma20,",3,0.008870,1,000078,2020-10-20,4.23,4.25,4.25,4.18,138903.20,0.01,0.24,4.280,4.261,4.232,223553.68,218116.87,228733.54,0.53,海王生物,医药商业,41.68,4.260000,4.465333
8,2020-10-16,"ma5,ma10,",2,0.005760,1,000166,2020-10-16,5.45,5.52,5.45,5.44,703478.00,0.01,0.18,5.484,5.440,5.375,757196.85,705822.16,728483.24,0.31,申万宏源,证券,15.64,5.437000,5.579833
10,2020-10-21,"ma5,ma10,ma20,ma30,ma60,",5,0.029883,1,000301,2020-10-21,6.53,6.60,6.54,6.43,54748.36,0.01,0.15,6.526,6.527,6.393,72678.23,91311.16,84013.51,0.43,东方盛虹,化纤,235.47,6.383333,6.103167
8,2020-10-16,"ma5,ma60,",2,0.002494,1,000404,2020-10-16,4.14,4.19,4.15,4.10,81205.00,0.00,0.00,4.104,4.012,4.041,73833.43,59342.95,63376.20,1.17,长虹华意,专用机械,27.25,4.132333,4.103000
8,2020-10-21,"ma5,ma10,",2,0.001761,2,000429,2020-10-21,6.84,6.90,6.90,6.82,11922.75,0.06,0.88,6.882,6.857,6.863,16864.49,17521.47,15549.48,0.09,粤高速Ａ,路桥,30.98,6.879667,6.961167
10,2020-10-22,"ma5,ma10,",2,0.001597,1,000514,2020-10-22,4.04,4.04,4.02,3.99,24228.83,0.00,0.00,4.044,4.017,4.004,56699.80,43066.75,41733.37,0.29,渝 开 发,区域地产,93.75,4.040333,4.107667


In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle(r"E:\work_dir\stock\src\stockProject\history_data\stock_2020-11-22.pkl")

In [17]:
df2 = df.query("name=='鲁商发展'").sort_values('date')

In [18]:
df2

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,symbol,name,industry,pe
469967,2018-11-15,3.13,3.13,3.07,3.05,31086.00,0.01,0.33,3.014,2.970,2.894,43317.67,30745.25,28141.24,0.31,600223,鲁商发展,区域地产,20.81
469966,2018-11-16,3.14,3.16,3.15,3.09,46578.77,0.08,2.61,3.060,2.993,2.918,47348.23,33187.28,29384.51,0.47,600223,鲁商发展,区域地产,20.81
469965,2018-11-19,3.17,3.27,3.21,3.17,49365.35,0.06,1.91,3.108,3.021,2.940,49324.27,36438.23,30502.92,0.49,600223,鲁商发展,区域地产,20.81
469964,2018-11-20,3.21,3.30,3.18,3.15,52297.51,-0.03,-0.94,3.134,3.047,2.962,44540.72,39895.00,32284.00,0.52,600223,鲁商发展,区域地产,20.81
469963,2018-11-21,3.15,3.23,3.22,3.14,21636.00,0.04,1.26,3.166,3.076,2.983,40192.73,40314.90,32512.66,0.22,600223,鲁商发展,区域地产,20.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9914,2020-11-16,12.16,12.85,12.42,12.16,494065.03,0.40,3.33,11.718,11.672,11.031,301252.50,355008.46,270382.71,4.94,600223,鲁商发展,区域地产,20.81
7881,2020-11-17,12.36,12.87,12.49,12.24,436036.56,0.07,0.56,11.942,11.839,11.122,327876.33,376050.65,284959.60,4.36,600223,鲁商发展,区域地产,20.81
6780,2020-11-18,12.35,12.68,12.15,11.95,394064.12,-0.34,-2.72,12.104,11.899,11.195,365266.39,362407.14,296336.68,3.94,600223,鲁商发展,区域地产,22.72
7619,2020-11-19,11.90,11.95,11.86,11.57,236987.95,-0.29,-2.39,12.188,11.865,11.263,381772.39,330267.68,302466.39,2.37,600223,鲁商发展,区域地产,22.72


In [27]:
%time df2['close'].ewm(span=30,ignore_na=False, adjust= True).mean()

Wall time: 995 µs


469967     3.070000
469966     3.111333
469965     3.146438
469964     3.155686
469963     3.170319
            ...    
9914      11.096570
7881      11.186469
6780      11.248632
7619      11.288075
8426      11.319812
Name: close, Length: 490, dtype: float64

In [58]:
cs = pd.read_csv(r"E:\work_dir\stock\src\stockProject\results\get_ma_A_ma30_stock.csv").sort_values('jx_date')

In [59]:
cs['jx_counts_30'] = (cs[['ma10','ma20','ma30']]>0).sum(axis=1)
# cs['jx_counts_60'] = (cs[['ma10','ma20','ma30','ma60']]>0).sum(axis=1)

In [61]:
cs[(cs['var']<=0.01)]

,jx_date,var,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,...,industry,pe,bf_close,af_close,sw_class,pord_5,pord_10,pord_15,pord_30,jx_counts_30
7,2018-12-20,0.0043,0.01,-0.01,0.01,0.00,3,"0.01,-0.01,0.01,0.0",600223,2018-12-20,...,区域地产,20.81,0.11,-0.15,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,1
8,2018-12-21,0.0067,0.02,-0.01,0.01,0.00,3,"0.02,-0.01,0.01,0.0",600223,2018-12-21,...,区域地产,20.81,0.04,-0.12,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,1
9,2018-12-24,0.0080,0.03,-0.01,0.01,0.00,3,"0.03,-0.01,0.01,0.0",600223,2018-12-24,...,区域地产,20.81,-0.02,-0.05,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,1
10,2018-12-25,0.0093,0.03,-0.00,0.01,0.00,3,"0.03,-0.0,0.01,0.0",600223,2018-12-25,...,区域地产,20.81,0.00,-0.04,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,1
11,2018-12-26,0.0050,0.01,-0.01,0.00,0.00,3,"0.01,-0.01,0.0,0.0",600223,2018-12-26,...,区域地产,20.81,-0.01,-0.02,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2020-10-26,0.0086,-0.03,0.04,0.01,-0.04,2,"-0.03,0.04,0.01,-0.04",600223,2020-10-26,...,区域地产,20.81,-0.10,0.10,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,2
102,2020-10-27,0.0073,-0.08,0.01,0.00,-0.04,2,"-0.08,0.01,0.0,-0.04",600223,2020-10-27,...,区域地产,20.81,-0.07,0.15,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,1
103,2020-10-30,0.0050,-0.05,-0.02,0.00,0.00,2,"-0.05,-0.02,0.0,0.0",600223,2020-10-30,...,区域地产,20.81,-0.01,0.13,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,0
104,2020-11-02,0.0053,0.02,-0.01,0.02,0.01,3,"0.02,-0.01,0.02,0.01",600223,2020-11-02,...,区域地产,20.81,0.04,0.08,get_ma_A_ma30,0.2778,0.2778,0.4118,0.5,2


In [57]:
cs[(cs['var']<=0.008)&(cs['jx_counts_30']>=2)]

,jx_date,var,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,...,industry,pe,bf_close,af_close,sw_class,pord_5,pord_10,pord_15,pord_30,jx_counts_30
0,2018-11-15,0.000000,1.53,1.53,1.53,1.53,4,"1.53,1.53,1.53,1.53",600223,2018-11-15,...,区域地产,20.81,NaN,0.03,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
1,2018-11-16,0.000696,0.05,0.04,0.04,0.04,4,"0.05,0.04,0.04,0.04",600223,2018-11-16,...,区域地产,20.81,NaN,0.01,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
2,2018-11-19,0.001566,0.05,0.04,0.04,0.04,4,"0.05,0.04,0.04,0.04",600223,2018-11-19,...,区域地产,20.81,NaN,0.00,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
3,2018-11-20,0.001419,0.03,0.03,0.03,0.03,4,"0.03,0.03,0.03,0.03",600223,2018-11-20,...,区域地产,20.81,NaN,0.02,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
4,2018-11-21,0.001917,0.03,0.03,0.03,0.02,4,"0.03,0.03,0.03,0.02",600223,2018-11-21,...,区域地产,20.81,NaN,0.00,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
5,2018-11-22,0.003287,0.03,0.03,0.02,0.02,4,"0.03,0.03,0.02,0.02",600223,2018-11-22,...,区域地产,20.81,NaN,-0.01,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
6,2018-11-23,0.001592,0.02,0.02,0.02,0.02,4,"0.02,0.02,0.02,0.02",600223,2018-11-23,...,区域地产,20.81,0.03,0.05,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
7,2018-11-26,0.000741,0.01,0.01,0.01,0.01,4,"0.01,0.01,0.01,0.01",600223,2018-11-26,...,区域地产,20.81,0.01,0.13,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
8,2018-11-27,0.001012,0.00,0.01,0.01,0.01,4,"0.0,0.01,0.01,0.01",600223,2018-11-27,...,区域地产,20.81,0.00,0.13,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3
12,2018-12-03,0.004406,0.01,0.01,0.01,0.01,4,"0.01,0.01,0.01,0.01",600223,2018-12-03,...,区域地产,20.81,0.05,-0.02,get_ma_A_ma30,0.2632,0.4211,0.3889,0.4118,3


In [39]:
cs.sw_class

0      get_ma30_A_ma60
1      get_ma30_A_ma60
2      get_ma30_A_ma60
3      get_ma30_A_ma60
4      get_ma30_A_ma60
            ...       
164    get_ma30_A_ma60
165    get_ma30_A_ma60
166    get_ma30_A_ma60
167    get_ma30_A_ma60
168    get_ma30_A_ma60
Name: sw_class, Length: 169, dtype: object